# Web Components

En el capítulo del DOM hemos introducido gran parte de los conocimientos necesarios para este capítulo. No obstante, necesitamos haber trabajado con todos los demás conceptos y haber practicado con proyectos simples para entender la importancia de los `Web Components`. 

Si queremos hacer una SPA que maneje datos obtenidos del servidor y los muestre de manera reactiva, podemos seguir estos pasos:

* El router manda descargar los datos del servidor y los pasa a una función que los aplica a una plantilla. 
* El router se suscribe a un Observable de los datos del servidor y cada vez que llegan nuevos datos los renderiza con una plantilla.
* El router renderiza una plantilla con "placeholders", espera los datos del servidor y rellena la plantilla con esos datos. 
* ...

En cualquiera de estas opciones, hay problemas para volver a renderizar cuando hay datos nuevos, para mantener a raya a las suscripciones, para gestionar el estado de la aplicación o para mantenerla suficientemente desacoplada. Además, será complicado reutilizar esas funciones en otros proyectos.  

Los `Web Components` no son una tecnología diferente a las vistas en los capítulos anteriores, se trata de una definición de la W3C para estandarizar lo que comenzaba a ser un aspecto común de los frameworks e intentar hacer componentes reutilizables para todos. Estos componentes se pueden hacer a partir de las mejoras de ES6 y HTML5. En la definición que podemos encontrar en la web de MDN: https://developer.mozilla.org/en-US/docs/Web/API/Web_components, no incorpora la reactividad, pero nosotros vamos a incorporarla desde el principio. 

> Llegado el momento de hacer Web Components desde 0 con "vanilla Javascript" nos podemos preguntar si no será mejor usar un framework. Intentemos resistir esa tentación, al menos para aprender, porque esos conocimientos nos permitirán entender profundamente cómo funciona, por ejemplo. Angular. 



## Conceptos iniciales

Se pueden crear componentes de muchas maneras, pero la definición oficial parte de la posibilidad de crear **Elementos personalizados** o `Custom Elements`. Esto se puede hacer porque Javascript permite heredar de `HTMLElement`: https://developer.mozilla.org/en-US/docs/Web/API/Web_components/Using_custom_elements. Luego veremos las posibilidades que hay con esto. 

Los elementos personalizados pueden ser diferentes al resto de la aplicación y tener sus propios scripts y estilos. Estos pueden colisionar con una aplicación para la que no fueron creados desde el principio. Una de las necesidades cubiertas por los Web Components es la reutilización del código. Pero Javascript permite encapsular estos elementos en un DOM separado del principal. Esta técnica se llama `Shadow DOM`. 

Para crear elementos personalizados en el `Shadow DOM`, se pueden hacer programáticamente o a través del HTML con etiquetas como `<template>` o `<slot>`. 

Por consiguiente, un `Web Component` tiene una clase que hereda de `HTMLElement` registrada en el `CustomElementRegistry` para que pueda ser utilizada en cualquier parte. Este elemento tiene un `Shadow DOM` (opcional) para que su código y estilos no molesten al resto y se suele usar `<template>` para las plantillas. A continuación se puede usar como cualquier etiqueta. 

### Alternativas

Como todo en Javascript, es opcional usar Web components, pero es que también es opcional usar todas las técnicas anteriores. Si no hacemos un `Custom Element` no entra en la definición de lo que estamos tratando, pero se puede hacer una función que retorne un elemento con `Shadow DOM` y código personalizado que atiende a los eventos, descarga los datos, se suscribe a Observables... El problema es que perdemos las ventajas del ciclo de vida de los `HTMLElements`. 

Tampoco es necesario siempre hacer `Shadow DOM` si el código o estilo no va a interferir nunca. 

Por otro lado, el uso de `<template>` puede ser muy farragoso en algunos casos comparado con las `Template Literals`. En nuestro caso, haremos un uso combinado de los mismos para aprovechar las ventajas de ambas técnicas. 

## Custom Elements

Se trata de elementos HTML que tienen un comportamiento definido por el desarrollador. Una vez registrados, quedan disponibles en el navegador en esa página web.

Ya existen algunos creados como `HTMLImageElement` o `HTMLParagrafElement` que tienen un comportamiento extendido respecto al estándar de los elementos comunes. Pero a nosotros nos interesa crear nuevos desde cero. 

Para crearlo tan solo hay que extender la clase:

```javascript
class PopupInfo extends HTMLElement {
  constructor() {
    super();
  }
  // Element functionality written in here
}
```

En el constructor se inicializa el elemento con valores por defecto, registro de eventos, creación del `Shadow Root` y poco más. Dejaremos la creación de elementos hijos o añadir atributos a estos para después, en otras etapas del ciclo de vida. 

### Ciclo de vida

Los elementos personalizados cuentan con una serie de funciones que pueden ser implementadas y que son invocadas a lo largo de su ciclo de vida. Estas son:

* connectedCallback(): Se llama cada vez que se agrega el elemento al documento. Se recomienda configurar el elemento en este punto, mejor que en el constructor. 
* disconnectedCallback(): Se llama cada vez que se elimina el elemento del documento.
* adoptedCallback(): Se llama cada vez que se mueve el elemento a un nuevo documento.
* attributeChangedCallback(): se llama cuando se cambian, agregan, eliminan o reemplazan atributos. 

Podemos probar este código en la consola del navegador: 

```javascript
class MyCustomElement extends HTMLElement {
  static observedAttributes = ["class", "size"];

  constructor() {
    super();
      console.log("Constructor")
  }

  connectedCallback() {
    console.log("Custom element added to page.");
  }

  disconnectedCallback() {
    console.log("Custom element removed from page.");
  }

  adoptedCallback() {
    console.log("Custom element moved to new page.");
  }

  attributeChangedCallback(name, oldValue, newValue) {
    console.log(`Attribute ${name} has changed from ${oldValue} to ${newValue}.`);
  }
}

customElements.define("my-custom-element", MyCustomElement);
// La prueba
let customElement = document.createElement('my-custom-element');
console.log('Creado');
document.body.append(customElement);
console.log('Añadido');
customElement.classList.add('customClass','customClass2');
console.log('Cambio del atributo class');
customElement.remove();
console.log('Eliminado');
```

### Registrar el elemento personalizado

Como se puede ver en el ejemplo anterior, lo hemos registrado con:

```javascript
customElements.define("my-custom-element", MyCustomElement);
```

Es importante recalcar que el nombre es preciso que tenga **un guión (-)** en medio del nombre para que Javascript no los confunda. 

### Usar el elemento personalizado

Se puede usar de forma normal como cualquier etiqueta HTML:

```html
<my-custom-element></my-custom-element>
```

## Shadow DOM

Como ya hemos visto en la introducción, el objetivo del `Shadow DOM` es encapsular el comportamiento del elemento personalizado. 

A partir de la raíz de DOM principal de la página web cuelgan todos los nodos de forma jerárquica. El Shadow DOM permite registrar árboles ocultos a partir de un nodo del árbol principal. Estos árboles empiezan por el `Shadow Root`. Esta raíz está unida al árbol principal mediante un nodo que actúa de `Shadow Host`. 

Se pueden crear de forma imperativa mediante Javascript:

```javascript
const host = document.querySelector("#host");
const shadow = host.attachShadow({ mode: "open" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM";
shadow.appendChild(span);
```

También se puede crear de forma declarativa mediante HTML:

```html
<div id="host">
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM</span>
  </template>
</div>
```

La segunda manera permite delegar la creación del `Shadow DOM` al servidor, al enviar este el HTML ya sea dinámica o estáticamente. 

Observemos que en los dos casos tenemos la palabra `open`, esta permite que el `Shadow DOM` sea accesible desde la web. Si no ponemos el `shadowrootmode` no se verá, si lo ponemos, ya sea open o close, se verá pero será o no accesible.  

Llegados a este punto, podemos poner esto en práctica copiando y pegando este código en la consola de cualquier web que tengamos abierta:


```javascript
const body = document.querySelector("body");
const host = document.createElement('div');
body.append(host);
const shadow = host.attachShadow({ mode: "closed" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM from Javascript";
shadow.append(span);

const host2 = document.createElement('div');
host2.innerHTML = `
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM from HTML</span>
  </template>
`;
body.append(host2);
```

El segundo no se va a ver porque innerHTML no crea el `Shadow Root`. Este, por javascript, ha de ser creado explícitamente como en el primer ejemplo. No obstante, aunque no se vea, el template queda accesible y puede ser clonado para incorporarlo más adelante. 





Lecturas:
* https://developer.mozilla.org/en-US/docs/Web/API/Web_components: Guia principal de referencia. 
* https://github.com/mdn/web-components-examples: Ejemplos de Custom Elements de la MDN.
* https://es.javascript.info/web-components Otro muy buen manual. 